#### Import packages

In [ ]:
import Pkg
Pkg.add("FastMarching")
Pkg.add("Gadfly")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("DifferentialEquations")
Pkg.add("Eikonal")
Pkg.add("Gridap")
Pkg.add("Optim")
Pkg.add("Cairo")
Pkg.add("Fontconfig") 

 First, we set up a grid. We propose a 1000m x 1000m grid with nodes separated by one meter in both directions. We therefore need 1001 nodes in each direction We will work in the SI to simplify the code and not make silly mistakes.

In [67]:
Nx=1001;
Ny=1001;

To solve the eikonal equation we need to model the wave speed. A simple model is  $c=c_{air}+\vec{v_{wind}}.\vec{n}$. Now we set $c_{air}=340 m/s$  and $v_{wind}=(10,0)m/s$. In the X direction, the wavefront normal is $\vec{n}=(1,0)$ and consequently $c_x=350 m/s$. In the Y direction, the wavefront normal is $\vec{n}=(0,1)$ and consequently $c_y=340 m/s$. In diagonal $\vec{n}=\frac{(1,1)}{\sqrt{2}}$ and $c_d=(340+\frac{10}{\sqrt{2}}) m/s= 347,0710678 m/s$

 We obtain the gradient of the flight time from the Eikonal equation.  ${\nabla{T}(x,y)}^2=\frac{1}{c(x,y)^2} \Rightarrow |\nabla{T}(x,y)|=|\frac{1}{c(x,y)}| $ . This leads to a system of partial differential equations. However, in certain directions where the speed is constant, we know the analytical result which is simply $\nabla{T}=1/c$.

We can analytically know the result for a given source and a given distance in certain constant speed scenarios, so it serves as a "benchmark" for the methods to be applied.

# FastMarching package

ref:https://docs.juliahub.com/General/FastMarching/stable/autodocs/

### Msfm function of FastMarching 

We see that flight time is obtained using the FastMarching package already implemented in Julia. First, we observe the result for a single constant speed in all directions that matches with c_air=340m/s.  We set $(x_i,y_i)=(0,0)m ; (x_f, y_f)=(1000,1000)m$. The distance travelled is $d=\sqrt{1000m^2+1000m^2}=1414,21m$

In [68]:
@time begin
using FastMarching
c_air=340.0
c_air_matrix= fill(c_air, (Nx, Ny))
SourcePoint = [1.,1.]  # Starting point
T = FastMarching.msfm(c_air_matrix, SourcePoint, true, true)
# We can obtain the flight time athe receiver [100m,100m]
travel_time = T[1001, 1001];
analytic_travel_time=sqrt(1000^2+1000^2)/c_air;
end
println("The travel time obtained by FastMarching is $travel_time")
println("The travel time obtained analytically is $analytic_travel_time")

  2.870004 seconds (338 allocations: 38.366 MiB)
The travel time obtained by FastMarching is 4.1595811072585
The travel time obtained analytically is 4.159451654038515


Now we must define the wave speed map at every node point based on the expression $c=c_{air}+\vec{v_{wind}}.\vec{n}$; with $\vec{n}$ variable depending on the point. 

In [69]:
using LinearAlgebra
function map_speed(c_air::Float64,v_wind::Array{Float64,1},SourcePoint::Array{Float64,1},Nx::Int64, Ny::Int64)
    c=fill(c_air, (Nx, Ny));
    n = zeros(Float64,Nx, Ny, 2);
    for i in 1:Nx, j in 1:Ny
        vector=[float(i),float(j)]-SourcePoint;
        n[i,j,:]=vector/norm(vector);
        c[i,j]=c_air+dot(v_wind,n[i,j,:]);
        if [float(i),float(j)]==SourcePoint 
            #Initial condition
            c[i,j]=c_air
            n[i,j,:]=[0.,0.]
        end  
    end
    return c
end

map_speed (generic function with 1 method)

In [70]:
v_wind=[10.,0.];
#Even though the source corresponds to nodes, it is simpler to define it as a vector of floating point components.
SourcePoint=[1.,1.];
c=map_speed(c_air,v_wind,SourcePoint,Nx,Ny);
#Plotting
using Plots
Plots.heatmap(transpose(c), xlims=(0,1000), ylims=(0,1000), title="Map of the speed of sound over the region", xlabel="x (m)", ylabel="y (m)")
savefig("MapavsonidoBenchmarking.png")

"C:\\Users\\Miguel\\Desktop\\TFM\\Codigos Julia\\MapavsonidoBenchmarking.png"

In [71]:
@time begin
v_wind=[10.,0.];
#Even though the source corresponds to nodes, it is simpler to define it as a vector of floating point components.
SourcePoint=[1.,1.];
c=map_speed(c_air,v_wind,SourcePoint,Nx,Ny);
T = FastMarching.msfm(c, SourcePoint, true, true)
println("Along the X axis:")
travel_time_X = T[1001, 1];
end 

analytic_travel_time_X=1000/350.
println("The travel time obtained by FastMarching is $travel_time_X")
println("The travel time obtained analytically is $analytic_travel_time_X")

println("Along the Y axis:")
travel_time_Y = T[1, 1001];
analytic_travel_time_Y=1000/340.
println("The travel time obtained by FastMarching is $travel_time_Y")
println("The travel time obtained analytically is $analytic_travel_time_Y")


Along the X axis:
  3.603463 seconds (5.01 M allocations: 435.867 MiB, 4.67% gc time)
The travel time obtained by FastMarching is 2.8571428571606927
The travel time obtained analytically is 2.857142857142857
Along the Y axis:
The travel time obtained by FastMarching is 2.940143116901547
The travel time obtained analytically is 2.9411764705882355


In the Y direction, a similar but different result is obtained. It may not follow a straight path in the Y direction. Makes sense.

# Gadfly package

In [72]:
using Gadfly
using Cairo
using Fontconfig
p=Gadfly.plot(z=T, Geom.contour,Coord.Cartesian(xmin=0, xmax=1000, ymin=0, ymax=1000), Guide.xlabel("x (m)"),
    Guide.ylabel("y (m)"),Guide.title("Fast Marching"),Guide.colorkey(title ="Traveltime (s)"))
Gadfly.push!(p, layer(x=[SourcePoint[1]], y=[SourcePoint[2]], Geom.point, Theme(default_color="red")))
draw(PNG("gadflyFM.png", 8inch, 6inch), p)

# Fast Sweeping method of Eikonal package

Ref: https://github.com/triscale-innov/Eikonal.jl/blob/main/docs/ripple-tank/ripple-tank.md

In [73]:
using Eikonal

# Define the slowness field sigma in the Eikonal context
@time begin
sigma = 1.0 ./ c;

fs = FastSweeping(sigma);

# Source point. We must use integers here
source_point = (1, 1);

# Initialize the wavefront arrival time at the source point
init!(fs, source_point);
sweep!(fs, verbose=true);
travel_time_FSM=fs.t[1:Nx,1:Ny];
travel_time_FSM_x=travel_time_FSM[1001,1];
end

#The time matrix fs.t is initialized with an additional dimension in each direction to handle boundary conditions.
#This is done to avoid out-of-range index problems during the sweeping calculation. 
#The additional values in fs.t represent the travel times in the “ghost cells” outside the real calculation domain.
#Therefore, fs.t is a matrix of dimension (Nx+1) x (Ny+1).



iter 1, sweep 1: change = 0.0
iter 1, sweep 2: change = Inf
iter 1, sweep 3: change = 0.0
iter 1, sweep 4: change = Inf
iter 2, sweep 1: change = Inf
iter 2, sweep 2: change = 0.0001767983918250674
iter 2, sweep 3: change = 0.0
iter 2, sweep 4: change = 0.0
iter 3, sweep 1: change = 3.6063349963160013e-6
iter 3, sweep 2: change = 3.1003804594091173e-6
iter 3, sweep 3: change = 0.0
iter 3, sweep 4: change = 0.0
iter 4, sweep 1: change = 9.58377603392923e-7
iter 4, sweep 2: change = 8.870204499939237e-7
iter 4, sweep 3: change = 0.0
iter 4, sweep 4: change = 0.0
iter 5, sweep 1: change = 3.1093367521076296e-7
iter 5, sweep 2: change = 2.9742935305226793e-7
iter 5, sweep 3: change = 0.0
iter 5, sweep 4: change = 0.0
iter 6, sweep 1: change = 1.0552134995493261e-7
iter 6, sweep 2: change = 1.0249527539862101e-7
iter 6, sweep 3: change = 0.0
iter 6, sweep 4: change = 0.0
iter 7, sweep 1: change = 3.706399961610193e-8
iter 7, sweep 2: change = 3.6360080460685386e-8
iter 7, sweep 3: change = 

2.857226890756263

In [74]:
println("The travel time obtained by FastSweeping along the X axis is $travel_time_FSM_x")
println("The travel time obtained analytically along the X axis is $analytic_travel_time_X")
using Gadfly
p2=Gadfly.plot(z=travel_time_FSM, Geom.contour,Coord.Cartesian(xmin=0, xmax=1000, ymin=0, ymax=1000), Guide.xlabel("x (m)"),
    Guide.ylabel("y (m)"),Guide.title("Fast Sweeping"),Guide.colorkey(title ="Traveltime (s)"))
Gadfly.push!(p2, layer(x=[SourcePoint[1]], y=[SourcePoint[2]], Geom.point, Theme(default_color="red")))
draw(PNG("gadflyFS.png", 8inch, 6inch), p2)

The travel time obtained by FastSweeping along the X axis is 2.857226890756263
The travel time obtained analytically along the X axis is 2.857142857142857


# DifferentialEquations package

In certain directions where the speed is constant we can use this package to solve $\frac{du}{dx}=f(u,p,x)$. In the X direction we can set c=350m/s.

Ref: https://docs.sciml.ai/DiffEqDocs/stable/

In [78]:
@time begin
using DifferentialEquations
# We define the ODE
f(u,p,x) = 1/350.;

# Initial condition
u0 = 0;
#Interval
xspan = (0.0, 1000.0);

# Solve the ODEProblem
prob = ODEProblem(f, u0, xspan);
sol = DifferentialEquations.solve(prob);
travel_time=sol(1000.);
end
analytical_travel_time=1000/350.
println("The travel time given by DifferentialEquations is $travel_time")
println("The travel time obtained analytically is $analytical_travel_time")


  1.112901 seconds (633.62 k allocations: 42.262 MiB, 99.82% compilation time: 100% of which was recompilation)
The travel time given by DifferentialEquations is 2.8571428571428434
The travel time obtained analytically is 2.857142857142857


In [79]:
@time begin
#Now we apply fourth order Runge-Kutta 
using DifferentialEquations

# We define the ODE
f(u,p,x) = 1/350;

# Initial condition
u0 = 0;
#Interval
xspan = (0.0, 1000.0);

# Solve the ODEProblem
prob = ODEProblem(f, u0, xspan);
sol = DifferentialEquations.solve(prob,RK4());
travel_time=sol(1000.);
end
analytical_travel_time=1000/350.
println("The travel time given by DifferentialEquations RK4 is $travel_time")
println("The travel time obtained analytically is $analytical_travel_time")

  0.628881 seconds (422.14 k allocations: 28.070 MiB, 99.78% compilation time: 100% of which was recompilation)
The travel time given by DifferentialEquations RK4 is 2.857142857142857
The travel time obtained analytically is 2.857142857142857
